<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/L_test/email-subject/model-tuning/Flan%20T5%20Base%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 183 (delta 86), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (183/183), 7.95 MiB | 3.93 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [2]:
!pip install transformers
!pip install evaluate
!pip install rouge-score # Installing rouge-score library (https://pypi.org/project/rouge-score/)

# !pip install accelerate -U

!pip install transformers[torch]
!pip install sacrebleu

!pip install datasets nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Foun

In [5]:
# Importing Libraries

import os
import re
import pandas as pd # Data Handling
import tensorflow as tf
import numpy as np
import torch
import time
import seaborn as sns

from datasets import load_metric
from datasets import Dataset
import evaluate

from google.colab import files
import shutil

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
!pip install nltk
import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

# **Load Dataset**

In [6]:
dataset_df = pd.read_csv('/content/qM-AI-L/email-subject/model-tuning/train_emails.csv')

In [7]:
dataset_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
0,allen-p_inbox_20,"Greg/Phillip, Attached is the Grande Communic...",Service Agreement,NaN,NaN,NaN,65,2,gregphillip attached is the grande communicati...
1,allen-p_inbox_28,Phillip & Keith Attached is the first draw re...,Bishops Corner,NaN,NaN,NaN,145,2,phillip keith attached is the first draw reque...
2,allen-p_inbox_63,Your Internet Banking accounts are now setup a...,Internet Banking,NaN,NaN,NaN,250,2,your internet banking accounts are now setup a...
3,allen-p_inbox_64,To our IBS Customers that are still hanging in...,Internet Banking,NaN,NaN,NaN,458,2,to our ibs customers that are still hanging in...
4,allen-p_inbox_65,Phillip Good Morning!\nI hope you had a wonder...,SMEs for expert stories,NaN,NaN,NaN,68,4,phillip good morning i hope you had a wonderfu...
...,...,...,...,...,...,...,...,...,...
14431,zufferli-j_inbox_43,This email is acknowledgement from the Power P...,Power Pool,NaN,NaN,NaN,227,2,this email is acknowledgement from the power p...
14432,zufferli-j_inbox_44,This email is acknowledgement from the Power P...,Power Pool of Alberta,NaN,NaN,NaN,277,4,this email is acknowledgement from the power p...
14433,zufferli-j_inbox_46,"John, Further to the voice message that I lef...",Enron Security,NaN,NaN,NaN,148,2,john further to the voice message that i left ...
14434,zufferli-j_inbox_8,Make sure that all curves are downloaded by th...,Simulation Curves,NaN,NaN,NaN,66,2,make sure that all curves are downloaded by th...


# **Test Train Split**

In [12]:
subset_df = dataset_df.sample(frac=0.3, random_state=42)
subset_df.shape

(4331, 9)

In [13]:
Train_df, Test_df = train_test_split(subset_df, test_size=0.2, random_state=42)

In [14]:
Train_df.shape, Test_df.shape

((3464, 9), (867, 9))

In [15]:
Train_dataset = Dataset.from_pandas(Train_df)
Test_dataset = Dataset.from_pandas(Test_df)

In [16]:
Train_dataset, Test_dataset

(Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 3464
 }),
 Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 867
 }))

**Create a subset of Dataset**

In [17]:
Train_dataset.shape, Test_dataset.shape

((3464, 10), (867, 10))

In [19]:
Train_dataset['cleaned_emails'][10]

'what do martinis jazz and george washington have in common fine arts friday a happy hour for professionals presented by the museum of fine arts houston conoco krts 92.1 friday march 1 2002 in the cafe express of the museums audrey jones beck building 5601 main street free admission 600 830 p.m. 5 martini specials complimentary hors doeuvres cash bar come as you are mix and mingle with other professionals and see the monumental lansdowne portrait of george washington by american painter gilbert stuart. now on view at the mfah the portrait made news in 2001 when the owner who had lent the painting to the national portrait gallery at the smithsonian institution since 1968 decided to sell it. the donald w. reynolds foundation of las vegas donated 30 million to allow the national portrait gallery to purchase the painting. the mfah is the first stop on the national tour george washington a national treasure.'

**Load the pre-trained FLAN-T5 models and its tokenizer directly from HuggingFace. FLAN-T5 was released in the paper Scaling Instruction-Finetuned Language Models - it is an enhanced version of T5 that has been finetuned in a mixture of tasks. Setting torch_dtype=torch.bfloat16 specifies the memory type to be used by this model.**

In [20]:
# Function to get size of models
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Trainable model parameters: {trainable_model_params}\nAll model parameters: {all_model_params}\nPercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



**Load Flan T5 Base Model**

In [21]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

print("For Model : FLAN-T5-Base \n",print_number_of_trainable_model_parameters(model))

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

For Model : FLAN-T5-Base 
 Trainable model parameters: 247577856
All model parameters: 247577856
Percentage of trainable model parameters: 100.00%


In [31]:
def preprocess_function(examples):
    inputs = [f"Generate a subject line for the following email: {email}" for email in examples['cleaned_emails']]
    targets = examples['subject']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the dataset
tokenized_train_dataset = Train_dataset.map(preprocess_function, batched=True, remove_columns=Train_dataset.column_names)
tokenized_eval_dataset = Test_dataset.map(preprocess_function, batched=True, remove_columns=Test_dataset.column_names)

Map:   0%|          | 0/3464 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

In [32]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3464
})

# **Test the Model with Zero Shot Inferencing**

**Using a pre-trained model from Hugging Face/sources - FLAN-T5-Base .**

In [25]:
index = 1

email = Train_dataset['cleaned_emails'][index]
subject = Train_dataset['subject'][index]

prompt = f"""
Generate a subject line for the following email.

Email:
{email}

Subject:
"""

inputs = tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(model.device) for k, v in inputs.items()}


output = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens=200)[0],skip_special_tokens=True)


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FLAN-T5 MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Generate a subject line for the following email.

Email:
sara further to my voicemail message of this afternoon attached is the merger letter. as it happens both chase and morgan had isda agreements in place. the letter selects the chase agreement which as you know we amended earlier this week as the preferred agreement and terminates the morgan agreement. please review and call me 2122702673 if you have any questions. if the letter is acceptable please call me and ill arrange for execution copies to be sent to your attention. thankyou again for you help.

Subject:

---------------------------------------------------------------------------------------------------
BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :
Merger Letter

---------------------------------------------------------------------------------------------------
FLAN-T5 MODEL GENERATION - ZERO SHOT SUBJECT LINE:

**Load Flan T5 Small Model**

In [ ]:
flan_t5_small_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
# flan_t5_small_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

print("For Model : Flan-t5-small \n",print_number_of_trainable_model_parameters(flan_t5_small_model))

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

For Model : Flan-t5-small 
 Trainable model parameters: 76961152
All model parameters: 76961152
Percentage of trainable model parameters: 100.00%


In [ ]:
# flan-t5-base vs. flan-t5-small parameters
247577856-76961152

170616704

**Using a pre-trained model from Hugging Face etc- FLAN-T5-SMALL .**

In [ ]:
inputs = flan_t5_base_tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# inputs = {k: v.to(device) for k, v in inputs.items()}


output = flan_t5_base_tokenizer.decode(
    flan_t5_small_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FLAN-T5-SMALL MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Generate the subject line for the following email.

Email:
many of you have asked about procedures to evaluate employee performance. we are finalizing a performance process designed to track and provide feedback on individual accomplishments for the benefit of each employee the company and the unsecured creditors committee. a very simple and straightforward online performance assessment system pas will allow us to automate parts of the process. pas will open next week on monday may 13 and remain open through friday may 24 2002. as an employee of a debtor company you should access the system and complete your self assessment by friday may 17 2002. your supervisor will review and approve submitted accomplishments and schedule a facetoface meeting to discuss your contributions offer feedback set priorities for the upcoming period discuss challenges and potential development o

**By testing with various models with the zero shot inferencing, we can see that the model struggles to extract the same subject line compared to the baseline subject, but it does pull out some important information from the email which indicates the models can be fine-tuned to the task at hand.**

# **Performing Full Fine-Tuning using FLAN-T5 BASE MODEL**

In [27]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=3,
    logging_dir='./logs',
    logging_steps=500,
)

In [33]:
# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,30.246200,25.399366


TrainOutput(global_step=866, training_loss=28.435623556581987, metrics={'train_runtime': 805.1976, 'train_samples_per_second': 4.302, 'train_steps_per_second': 1.076, 'total_flos': 2371999468879872.0, 'train_loss': 28.435623556581987, 'epoch': 1.0})

In [34]:
# Save the model and tokenizer
model.save_pretrained('./flan-t5-email-subject-generator')
tokenizer.save_pretrained('./flan-t5-email-subject-generator')

('./flan-t5-email-subject-generator/tokenizer_config.json',
 './flan-t5-email-subject-generator/special_tokens_map.json',
 './flan-t5-email-subject-generator/spiece.model',
 './flan-t5-email-subject-generator/added_tokens.json',
 './flan-t5-email-subject-generator/tokenizer.json')

In [49]:
# Load the fine-tuned model and tokenizer
model_path = './flan-t5-email-subject-generator'

finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
finetuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [44]:
def generate_subject(email, max_length=128):
    prompt = f"Generate a subject line for the following email: {email}"
    inputs = tokenizer(prompt, return_tensors='pt', max_length=512, truncation=True, padding=True)
    outputs = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    subject = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return subject

In [54]:
emails_for_scores_df = Test_df.sample(frac=0.1, random_state=42)
emails_for_scores_df.shape

(87, 9)

In [55]:
original_subjects = []
flan_T5_subjects = []
finetuned_flan_T5_subjects = []


for index, row in emails_for_scores_df.iterrows():
    email = row['cleaned_emails']
    original_subject = row['subject']
    prompt = f"""
    Generate a subject line for the following email.

    Email:
    {email}

    Subject:
    """

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    finetuned_model.to(device)

    original_model_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    finetuned_model_input_ids = finetuned_tokenizer(prompt, return_tensors="pt").input_ids.to(device)


    original_model_outputs = model.generate(input_ids=original_model_input_ids, num_beams=4,generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    flan_T5_subjects.append(original_model_text_output)

    instruct_model_outputs = finetuned_model.generate(input_ids=finetuned_model_input_ids, num_beams=4,generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = finetuned_tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    finetuned_flan_T5_subjects.append(instruct_model_text_output)

    original_subjects.append(original_subject)

zipped_subjects = list(zip(original_subjects, flan_T5_subjects, finetuned_flan_T5_subjects))

Flan_T5_df = pd.DataFrame(zipped_subjects, columns = ['True_subjects', 'Flan_T5_subjects', 'Finetuned_flan_T5_subjects'])
Flan_T5_df

,True_subjects,Flan_T5_subjects,Finetuned_flan_T5_subjects
0,Houston City Officials Reception July 31,rsvp lists for tonight's enronazurix20 city of...,rsvp lists for tonight's enronazurix20 city of...
1,ENA Trading Track,Ena Trading Track,Ena Trading Track
2,Vandalism in the village,ccv vandalism in ccv,ccv vandalism in ccv
3,Plano vs Homestead,new plano and revised homestead numbers,new plano and revised homestead numbers
4,BC Gas Amendment,bc gas isda,bc gas isda
...,...,...,...
82,You Did a Good Job,Dear Mr. Lay - Thank you for all your hard wor...,Dear Mr. Lay - I have appreciated working for ...
83,Life Insurance Premium Payment,Enron,Greetings from Turkey
84,Northwest Foreign Key Problems,Foreign Key Violations,Northwest - Foreign Key Violations
85,Revised Salmon Agreement,Enron/Salmon Stabilizer,Enron/Salmon/Stabilizer


In [56]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

print('FLAN T5 Base Model:')
print(original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL:')
print(finetuned_model_results)

FLAN T5 Base Model:
{'rouge1': 0.22891606787951266, 'rouge2': 0.08689272140182334, 'rougeL': 0.22145240330063287, 'rougeLsum': 0.22047920417875494}

FINETUNED FLAN T5 BASE MODEL:
{'rouge1': 0.25632736418088475, 'rouge2': 0.1103817669225346, 'rougeL': 0.24649278918191683, 'rougeLsum': 0.24538567634951952}


In [57]:

print("Absolute percentage improvement of FINETUNED MODEL over PRETRAINED")

improvement = (np.array(list(finetuned_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(finetuned_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of FINETUNED MODEL over PRETRAINED
rouge1: 2.74%
rouge2: 2.35%
rougeL: 2.50%
rougeLsum: 2.49%


In [58]:
!pip install nltk

import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Download the 'punkt' resource
nltk.download('punkt')

nltk.download('wordnet') # Download the WordNet corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [74]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [75]:
# Initialize metric scorers
rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = rouge_scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([word_tokenize(true_subject)], word_tokenize(generated_subject))

    # SACREBLEU
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    # BLEU
    bleu_score = sentence_bleu([word_tokenize(true_subject)], word_tokenize(generated_subject))


    return rougeL, meteor, sacre_bleu_score, bleu_score

# Apply metrics evaluation
Flan_T5_df['rougeL'], Flan_T5_df['meteor'], Flan_T5_df['sacrebleu'], Flan_T5_df['bleu'] = zip(*Flan_T5_df.apply(
    lambda row: evaluate_metrics(row['True_subjects'], row['Flan_T5_subjects']), axis=1))

# Calculate average scores
average_rougeL = Flan_T5_df['rougeL'].mean()
average_meteor = Flan_T5_df['meteor'].mean()
average_sacrebleu = Flan_T5_df['sacrebleu'].mean()
average_bleu = Flan_T5_df['bleu'].mean()


print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')
print(f'Average BLEU: {average_bleu:.4f}')


Average ROUGE-L: 0.2209
Average METEOR: 0.1709
Average SACREBLEU: 2.0642
Average BLEU: 0.0000


In [78]:
# Initialize metric scorers
from rouge_score import rouge_scorer
rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = rouge_scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([word_tokenize(true_subject)], word_tokenize(generated_subject))

    # SACREBLEU
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    # BLEU
    bleu_score = sentence_bleu([word_tokenize(true_subject)], word_tokenize(generated_subject))


    return rougeL, meteor, sacre_bleu_score, bleu_score

# Apply metrics evaluation
Flan_T5_df['rougeL'], Flan_T5_df['meteor'], Flan_T5_df['sacrebleu'], Flan_T5_df['bleu'] = zip(*Flan_T5_df.apply(
    lambda row: evaluate_metrics(row['True_subjects'], row['Finetuned_flan_T5_subjects']), axis=1))

# Calculate average scores
average_rougeL = Flan_T5_df['rougeL'].mean()
average_meteor = Flan_T5_df['meteor'].mean()
average_sacrebleu = Flan_T5_df['sacrebleu'].mean()
average_bleu = Flan_T5_df['bleu'].mean()


print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')
print(f'Average BLEU: {average_bleu:.4f}')

Average ROUGE-L: 0.2467
Average METEOR: 0.2016
Average SACREBLEU: 2.9431
Average BLEU: 0.0000


# **Perform Parameter Efficient Fine-Tuning (PEFT)**
Parameter Efficient Fine-Tuning (PEFT), which is more efficient than full fine-tuning and yields comparable results. PEFT, often referring to Low-Rank Adaptation (LoRA), enables fine-tuning with fewer compute resources, often a single GPU.

LoRA produces a small adapter (a few MBs) while keeping the original LLM unchanged. During inference, this adapter is combined with the original LLM, allowing multiple adapters to reuse the same LLM and reducing memory requirements for various tasks.

## **Setup the PEFT/LoRA model for Fine-Tuning**



In [79]:
!pip install \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.0 MB/s eta 0:00:00


In [80]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [82]:
peft_model = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

Trainable model parameters: 3538944
All model parameters: 251116800
Percentage of trainable model parameters: 1.41%
